# Transport/vehicle booking system

 Import Libraries and Setup

Imports: pandas (dataframes), numpy (numerics), faker (fake data), random, datetime.

fake = Faker() initializes the Faker object for name/city/random-generation.

In [8]:
# Import necessary libraries
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta

# Initialize Faker generator
fake = Faker()


Define Helper Functions and Fixed Sets

insert_missing_values randomly replaces a percentage of values with None.

Prepares lookup lists for categories, cities, routes, vehicle types, and statuses.

In [9]:
# Helper function to randomly insert None (missing) values into a Series
def insert_missing_values(series, missing_pct):
    # Always work on a copy and use .iloc (safe for both Series/DataFrames)
    series = series.copy()
    n_missing = int(len(series) * missing_pct)
    if n_missing == 0:
        return series
    missing_indices = random.sample(range(len(series)), n_missing)
    # Assign using .iloc which works with positional indices
    series.iloc[missing_indices] = None
    return series


# Value lists for ordinal/categorical columns
genders = ['Male', 'Female', 'Other']
age_groups = ['Child', 'Teen', 'Adult', 'Senior']
vehicle_types = ['Bus', 'Train', 'Van']
cities = [fake.city() for _ in range(20)]
routes = [f"Route {chr(65+i)}" for i in range(10)]
statuses = ['Confirmed', 'Cancelled', 'Pending']


Generate Passengers Table

600 passengers with random realistic attributes.

Weighted age groups (most are adults).

city and gender columns contain some missing values for realism.

In [10]:
# Number of passengers
n_passengers = 600

# Generate passenger data
passenger_data = {
    "passenger_id": range(1, n_passengers + 1),
    "name": [fake.name() for _ in range(n_passengers)],
    "gender": [random.choice(genders) for _ in range(n_passengers)],
    "age_group": [random.choices(age_groups, weights=[0.15,0.15,0.55,0.15])[0] for _ in range(n_passengers)],
    "registration_date": [fake.date_between(start_date="-4y", end_date="today") for _ in range(n_passengers)],
    "city": [random.choice(cities) for _ in range(n_passengers)],
}

passengers_df = pd.DataFrame(passenger_data)

# Insert ~5% missing values in 'city' and 'gender'
passengers_df['city'] = insert_missing_values(passengers_df['city'], 0.05)
passengers_df['gender'] = insert_missing_values(passengers_df['gender'], 0.04)

# Preview
passengers_df.head()



,passenger_id,name,gender,age_group,registration_date,city
0,1,John Turner,Male,Adult,2023-07-03,Allisonshire
1,2,Cheryl Arnold,Other,Adult,2024-05-27,North Michael
2,3,Sherry Williams,Female,Adult,2022-02-06,West Kathrynstad
3,4,Michelle Alvarez,Male,Adult,2024-10-31,Allisonshire
4,5,Natasha Wells,Male,Senior,2023-01-26,North Timothy


Generate Vehicles Table

40 vehicles of various types, capacities, years.

Random condition ratings; about 10% missing values for condition_rating.



In [11]:
# Number of vehicles
n_vehicles = 40

vehicle_data = {
    "vehicle_id": range(1, n_vehicles + 1),
    "vehicle_type": [random.choice(vehicle_types) for _ in range(n_vehicles)],
    "capacity": [random.choice([20, 30, 40, 50, 70, 100]) for _ in range(n_vehicles)],
    "registration_year": [random.randint(2010, 2023) for _ in range(n_vehicles)],
    "condition_rating": [random.randint(1, 5) for _ in range(n_vehicles)],
    "assigned_route": [random.choice(routes) for _ in range(n_vehicles)],
}

vehicles_df = pd.DataFrame(vehicle_data)

# Insert ~10% missing 'condition_rating'
vehicles_df['condition_rating'] = insert_missing_values(vehicles_df['condition_rating'], 0.10)

vehicles_df.head()


,vehicle_id,vehicle_type,capacity,registration_year,condition_rating,assigned_route
0,1,Bus,50,2013,NaN,Route A
1,2,Train,50,2020,1.0,Route B
2,3,Bus,20,2014,5.0,Route G
3,4,Van,50,2015,4.0,Route J
4,5,Bus,30,2015,2.0,Route E


Generate Bookings Table

Each booking links to an existing passenger and vehicle by ID.

Various trip dates within last year, random seat numbers, realistic ticket prices.

Satisfaction and status each have some missing values.

No duplicates: Each (booking_id, passenger_id) pair is unique.

In [12]:
# Number of bookings
n_bookings = 1200

booking_ids = range(1, n_bookings + 1)
passenger_choices = random.choices(passengers_df['passenger_id'], k=n_bookings)
vehicle_choices = random.choices(vehicles_df['vehicle_id'], k=n_bookings)

booking_data = {
    "booking_id": booking_ids,
    "passenger_id": passenger_choices,
    "vehicle_id": vehicle_choices,
    "trip_date": [fake.date_between(start_date="-1y", end_date="today") for _ in range(n_bookings)],
    "route": [random.choice(routes) for _ in range(n_bookings)],
    "seat_number": [random.randint(1, 100) for _ in range(n_bookings)],
    "ticket_price": [round(random.uniform(15.0, 120.0), 2) for _ in range(n_bookings)],
    "satisfaction_rating": [random.randint(1, 5) for _ in range(n_bookings)],
    "status": [random.choice(statuses) for _ in range(n_bookings)]
}

bookings_df = pd.DataFrame(booking_data)

# Insert 3% missing 'status', 2% missing 'satisfaction_rating'
bookings_df['status'] = insert_missing_values(bookings_df['status'], 0.03)
bookings_df['satisfaction_rating'] = insert_missing_values(bookings_df['satisfaction_rating'], 0.02)

bookings_df.head()


,booking_id,passenger_id,vehicle_id,trip_date,route,seat_number,ticket_price,satisfaction_rating,status
0,1,543,19,2025-03-26,Route E,18,105.72,4.0,Confirmed
1,2,54,11,2025-10-24,Route D,97,115.44,2.0,Confirmed
2,3,522,3,2025-03-19,Route C,85,119.71,1.0,Cancelled
3,4,67,30,2025-05-25,Route F,61,100.23,2.0,Pending
4,5,391,7,2025-06-05,Route C,81,51.55,4.0,Pending


Export to CSV for Easy SQLite Import

In [13]:
passengers_df.to_csv("passengers.csv", index=False)
vehicles_df.to_csv("vehicles.csv", index=False)
bookings_df.to_csv("bookings.csv", index=False)


In [14]:
# Check for missing values in Passengers table
print("Missing values in Passengers table:")
print(passengers_df.isnull().sum())
print()

# Check for missing values in Vehicles table
print("Missing values in Vehicles table:")
print(vehicles_df.isnull().sum())
print()

# Check for missing values in Bookings table
print("Missing values in Bookings table:")
print(bookings_df.isnull().sum())


Missing values in Passengers table:
passenger_id          0
name                  0
gender               24
age_group             0
registration_date     0
city                 30
dtype: int64

Missing values in Vehicles table:
vehicle_id           0
vehicle_type         0
capacity             0
registration_year    0
condition_rating     4
assigned_route       0
dtype: int64

Missing values in Bookings table:
booking_id              0
passenger_id            0
vehicle_id              0
trip_date               0
route                   0
seat_number             0
ticket_price            0
satisfaction_rating    24
status                 36
dtype: int64
